In [ ]:
# sentiment analysis
# positive vs negative verbal statement based on sentiment of post

# use packages that analyze the writing level - is there a difference in level of intelligence of dif people on dif subreddits?
# might have dif viewpoints in each subreddit - try to do some tagging of dif viewpoints in each subreddit; averaging of snetiment and averaging of level of intelligence
# to do tagging of dif viewpoints - based on sentiment?
# length of post

In [ ]:
# word tokenizer
# then use clustering advanced (go into three dots - there's a bunch in that folder)
    # clustering k-means lab
    

In [113]:
import pickle

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import csv, re

import nltk
import string
from html.parser import HTMLParser

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, fetch_mldata
from sklearn.utils import shuffle

# see https://ipython.readthedocs.io/en/stable/interactive/magics.html
%pylab inline

# sets backend to render higher res images
%config InlineBackend.figure_formats = ['retina']




# gensim
from gensim import corpora, models, similarities, matutils

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator


Populating the interactive namespace from numpy and matplotlib


/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shuffle']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [114]:
plt.rcParams['figure.figsize'] = [6,6]
sns.set_style("whitegrid")
sns.set_context("talk")

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/laurenfinkelstein/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
# load in data

corpus = pickle.load(open('reproductive_rights_corpus_cleaned_preprocessed.pkl', 'rb'))

In [4]:
corpus

['child deserve love want right force woman gestate birth child want care forthe good way reduce demand abortion make birth control free available antiabortion law make illegal abortion common lead dead pregnant woman believe fantasy think',
 'yes',
 'definitely agree statement',
 'prochoice exfoster kid exact childhood feel deeply',
 '  cheap compare state pay raise kid probably cost like   year',
 'abortion publicly fund stop',
 'child bear save money eh love anyways',
 'hello personally pro life advocate sway issue like hear argument just liberal town just curious point civil downvote immediately just try hear thank',
 'necessarily',
 'kind sick hear unwanted kid grow know love care million kid unwanted pregnancy grow unloved pretending need allow abortion unwanted kid neglect blatantly false need allow abortion woman deserve choose carry raise baby just want mean turn horrible abusive mother',
 'bot bleep bloop link thread place rprolife prochoice value money human nbspif follow li

# Vectorize

### TFIDF Vectorizer

In [25]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=20000)
#     stop_words=None

X_wv = word_vectorizer.fit_transform(corpus)

# char_vectorizer = TfidfVectorizer(
#     sublinear_tf=True,
#     strip_accents='unicode',
#     analyzer='char',
#     ngram_range=(1, 6),
#     max_features=30000)
# char_vectorizer.fit(df.stemmed)

In [26]:
X_wv

<1294x5178 sparse matrix of type '<class 'numpy.float64'>'
	with 25487 stored elements in Compressed Sparse Row format>

In [ ]:
# gets the names of the features, which goes into the dataframe in the next line of code

word_vectorizer.get_feature_names()

In [27]:
tfidf_vecs = pd.DataFrame(X_wv.toarray(), index=corpus, columns=word_vectorizer.get_feature_names())

In [28]:
tfidf_vecs.head()

,ab,abandon,abby,abdominal,abhorrent,abide,ability,able,abnegation,abnormal,...,yr,yup,z,zero,zodiac,zygote,zygoteeh,zygotei,zygoteperson,zygoteso
child deserve love want right force woman gestate birth child want care forthe good way reduce demand abortion make birth control free available antiabortion law make illegal abortion common lead dead pregnant woman believe fantasy think,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
definitely agree statement,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
prochoice exfoster kid exact childhood feel deeply,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cheap compare state pay raise kid probably cost like year,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Count Vectorizer

In [23]:
# incorporate stop words when creating the count vectorizer
# cv = CountVectorizer(stop_words = 'english')
cv = CountVectorizer()
X_cv = cv.fit_transform(corpus)
count_vecs = pd.DataFrame(X_cv.toarray(), index=corpus, columns=cv.get_feature_names())

In [24]:
count_vecs.head()

,ab,abandon,abby,abdominal,abhorrent,abide,ability,able,abnegation,abnormal,...,youyour,yr,yup,zero,zodiac,zygote,zygoteeh,zygotei,zygoteperson,zygoteso
child deserve love want right force woman gestate birth child want care forthe good way reduce demand abortion make birth control free available antiabortion law make illegal abortion common lead dead pregnant woman believe fantasy think,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
definitely agree statement,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
prochoice exfoster kid exact childhood feel deeply,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cheap compare state pay raise kid probably cost like year,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Topic Modeling

## Latent Semantic Analysis (LSA)

> Note: LSA is SVD applied to NLP

### LSA using TfidfVectorizer

In [49]:
lsa = TruncatedSVD(2) # I chose two topics because I'm looking for prochoice vs. prolife groups
topics_tfidf = lsa.fit_transform(tfidf_vecs)
lsa.explained_variance_ratio_

# first topic captures 0.6%, second topic captures 1.9%

array([0.00614714, 0.01946548])

The U matrix shows us the 2 resulting topics, and the terms that are associated with each topic. In this case:
- Component 1 (topic 1) seems to be about ?
- Component 2 (topic 2) seems to be about ?

In [52]:
# here we are creating topic to feature; relationship

topic_word_tfidf = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2"],
             columns = word_vectorizer.get_feature_names())
topic_word_tfidf

,ab,abandon,abby,abdominal,abhorrent,abide,ability,able,abnegation,abnormal,...,yr,yup,z,zero,zodiac,zygote,zygoteeh,zygotei,zygoteperson,zygoteso
component_1,0.001,0.005,0.001,0.002,0.002,0.001,0.02,0.029,0.0,0.001,...,0.001,0.001,0.002,0.007,0.002,0.014,0.001,0.002,0.003,0.003
component_2,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.00,-0.000,-0.0,-0.000,...,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000


In [37]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [53]:
display_topics(lsa, cv.get_feature_names(), 5)


Topic  0
saidthe, abortion, perception, scary, keen

Topic  1
deliciousif, potentialif, really, resolve, absolutely


The Vt matrix shows us the documents we started with, and how each document is made up of the 2 resulting topics. In this case:
- The first four documents seem to be about sports
- The last three documents seem to be about music

In [67]:
Vt_tfidf = pd.DataFrame(topics_tfidf.round(5),
             index = corpus,
             columns = ["component_1","component_2" ])
Vt_tfidf.head()

,component_1,component_2
child deserve love want right force woman gestate birth child want care forthe good way reduce demand abortion make birth control free available antiabortion law make illegal abortion common lead dead pregnant woman believe fantasy think,0.31727,-0.00203
yes,0.04905,0.00302
definitely agree statement,0.06207,-0.00038
prochoice exfoster kid exact childhood feel deeply,0.05066,-0.00035
cheap compare state pay raise kid probably cost like year,0.08791,-0.00062


##### cosine similarity

> Looking at cosine similarity, we can see that two documents both high in component 2 have a similarity of ?, and two documents high in different components will have a similarity of ?.

In [71]:
cosine_similarity((Vt_tfidf.iloc[0], Vt_tfidf.iloc[1])).round()

array([[1., 1.],
       [1., 1.]])

In [72]:
cosine_similarity((Vt_tfidf.iloc[0], Vt_tfidf.iloc[6])).round()

array([[1., 1.],
       [1., 1.]])

### LSA using Count Vectorizer

In [61]:
lsa = TruncatedSVD(2) # I chose two topics because I'm looking for prochoice vs. prolife groups
lsa_cv = lsa.fit_transform(count_vecs)
lsa.explained_variance_ratio_

# first topic captures 9.5%, second topic captures 4%

array([0.09553336, 0.04042564])

The U matrix shows us the 2 resulting topics, and the terms that are associated with each topic. In this case:
- Component 1 (topic 1) seems to be about ?
- Component 2 (topic 2) seems to be about ?

In [63]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2"],
             columns = cv.get_feature_names())
topic_word

,ab,abandon,abby,abdominal,abhorrent,abide,ability,able,abnegation,abnormal,...,youyour,yr,yup,zero,zodiac,zygote,zygoteeh,zygotei,zygoteperson,zygoteso
component_1,0.001,0.004,0.000,0.001,0.000,0.0,0.016,0.022,0.0,0.000,...,0.0,0.0,0.001,0.002,0.001,0.005,0.0,0.0,0.0,0.001
component_2,-0.000,-0.006,-0.001,-0.001,-0.001,-0.0,0.003,-0.011,0.0,-0.001,...,-0.0,-0.0,-0.000,-0.002,-0.000,-0.000,-0.0,-0.0,-0.0,-0.000


In [65]:
display_topics(lsa, cv.get_feature_names(), 10)


Topic  0
human, right, person, people, fetus, abortion, say, think, argument, make

Topic  1
human, right, person, fetus, treatment, jew, value, nazi, medical, intrinsic


The Vt matrix shows us the documents we started with, and how each document is made up of the 2 resulting topics. In this case:
- The first four documents seem to be about ?
- The last three documents seem to be about ?

In [68]:
Vt_cv = pd.DataFrame(lsa_cv.round(5),
             index = corpus,
             columns = ["component_1","component_2" ])
Vt_cv.head()

,component_1,component_2
child deserve love want right force woman gestate birth child want care forthe good way reduce demand abortion make birth control free available antiabortion law make illegal abortion common lead dead pregnant woman believe fantasy think,2.67801,-1.77887
yes,0.01586,-0.00669
definitely agree statement,0.07340,-0.05156
prochoice exfoster kid exact childhood feel deeply,0.12403,-0.09685
cheap compare state pay raise kid probably cost like year,0.23760,-0.17322


##### cosine similarity

> Looking at cosine similarity, we can see that two documents both high in component 2 have a similarity of ?, and two documents high in different components will have a similarity of ?.

In [69]:
cosine_similarity((Vt_cv.iloc[0], Vt_cv.iloc[1])).round()

array([[1., 1.],
       [1., 1.]])

In [70]:
cosine_similarity((Vt_cv.iloc[0], Vt_cv.iloc[6])).round()

array([[1., 1.],
       [1., 1.]])

## Non-Negative Matrix Factorization (NMF)

### NMF using TFIDF

In [74]:
nmf_model_tfidf = NMF(7)
doc_topic_nmf_tfidf = nmf_model_tfidf.fit_transform(X_wv)

The W matrix shows us the 2 resulting topics, and the terms that are associated with each topic. In this case:
- Component 1 (topic 1) seems to be about ?
- Component 2 (topic 2) seems to be about ?

In [75]:
topic_word_nmf_tfidf = pd.DataFrame(nmf_model_tfidf.components_.round(3),
             index = ["component_1","component_2"],
             columns = word_vectorizer.get_feature_names())
topic_word_nmf_tfidf

,ab,abandon,abby,abdominal,abhorrent,abide,ability,able,abnegation,abnormal,...,yr,yup,z,zero,zodiac,zygote,zygoteeh,zygotei,zygoteperson,zygoteso
component_1,0.002,0.012,0.002,0.005,0.004,0.002,0.046,0.068,0.001,0.003,...,0.003,0.002,0.004,0.017,0.004,0.032,0.002,0.004,0.006,0.007
component_2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [76]:
display_topics(nmf_model_tfidf, word_vectorizer.get_feature_names(), 10)

# Note - if you need more than 25 words to figure out the pattern, your topic buckets may be too big
# might have independent subthemes living in the buckets; need smaller more tailored buckets


Topic  0
s, abortion, people, say, just, life, right, child, think, woman

Topic  1
delete, post, reading, research, absolutely, sub, lot, word, yes, abortion


The H matrix shows us the documents we started with, and how each document is made up of the 2 resulting topics. In this case:
- The first document seems to be about music
- The last document seems to be about sports
- Everything in between is a combination of the two

In [78]:
ex_label = [c[:30]+"..." for c in corpus]

H = pd.DataFrame(doc_topic_nmf_tfidf.round(5),
             index = ex_label,
             columns = ["component_1","component_2" ])
H.head()

# things don't necessary add up to 1 or the same thing
# some are more generic discussions that don't necessarily have a strong tie in with music or sports
# could also generate a document with strong tie in to both

,component_1,component_2
child deserve love want right ...,0.13603,0.00000
yes...,0.02103,0.00143
definitely agree statement...,0.02661,0.00000
prochoice exfoster kid exact c...,0.02172,0.00000
cheap compare state pay rais...,0.03769,0.00000


### NMF using Count Vectorizer

In [89]:
nmf_model_cv = NMF(2)
doc_topic_nmf_cv = nmf_model_cv.fit_transform(X_cv)

The W matrix shows us the 2 resulting topics, and the terms that are associated with each topic. In this case:
- Component 1 (topic 1) seems to be about ?
- Component 2 (topic 2) seems to be about ?

In [90]:
topic_word_nmf_cv = pd.DataFrame(nmf_model_cv.components_.round(3),
             index = ["component_1","component_2"],
             columns = cv.get_feature_names())
topic_word_nmf_cv

,ab,abandon,abby,abdominal,abhorrent,abide,ability,able,abnegation,abnormal,...,youyour,yr,yup,zero,zodiac,zygote,zygoteeh,zygotei,zygoteperson,zygoteso
component_1,0.006,0.00,0.000,0.000,0.000,0.000,0.124,0.098,0.001,0.000,...,0.002,0.000,0.003,0.007,0.003,0.032,0.0,0.001,0.002,0.004
component_2,0.004,0.05,0.003,0.011,0.003,0.001,0.076,0.161,0.000,0.003,...,0.004,0.001,0.003,0.018,0.005,0.024,0.0,0.003,0.002,0.007


In [91]:
display_topics(nmf_model_cv, cv.get_feature_names(), 10)

# Note - if you need more than 25 words to figure out the pattern, your topic buckets may be too big
# might have independent subthemes living in the buckets; need smaller more tailored buckets


Topic  0
human, right, person, fetus, argument, say, think, treatment, medical, make

Topic  1
people, abortion, life, baby, child, woman, just, sex, pregnancy, say


The H matrix shows us the documents we started with, and how each document is made up of the 2 resulting topics. In this case:
- The first document seems to be about ?
- The last document seems to be about ?
- Everything in between is a combination of the two

In [88]:
H = pd.DataFrame(doc_topic_nmf_cv.round(5),
             index = ex_label,
             columns = ["component_1","component_2" ])
H.head()

# things don't necessary add up to 1 or the same thing
# some are more generic discussions that don't necessarily have a strong tie in with music or sports
# could also generate a document with strong tie in to both

,component_1,component_2
child deserve love want right ...,0.04427,0.44523
yes...,0.00058,0.00212
definitely agree statement...,0.00039,0.01283
prochoice exfoster kid exact c...,0.00018,0.02289
cheap compare state pay rais...,0.00281,0.04117


## Latent Dirichlet Allocation (LDA)

We'll try out [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) in [gensim](http://radimrehurek.com/gensim/index.html) 

### LDA with TFIDF

### LDA with Count Vectorizer

In [101]:
# create doc-term matrix and transpose it so terms are in rows

doc_word = cv.transform(count_vecs ).transpose()

In [102]:
pd.DataFrame(doc_word.toarray(), cv.get_feature_names()).head()

,0,1,2,3,4,5,6,7,8,9,...,5151,5152,5153,5154,5155,5156,5157,5158,5159,5160
ab,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abandon,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abby,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abdominal,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abhorrent,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
doc_word.shape

(5161, 5161)

In [105]:
# convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_word)

In [106]:
# map matrix rows to words (tokens)
# we need to save a mapping (dict) of row id to word (token) for later use by gensim:

id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [107]:
len(id2word)

5161

In [108]:
# Create lda model (equivalent to "fit" in sklearn)
# LDA requires our corpus of word counts, mapping of row ids to words, and the number of topics

lda = models.LdaModel(corpus=corpus, num_topics=2, id2word=id2word, passes=5)

2019-05-21 13:56:38,691 : INFO : using symmetric alpha at 0.5
2019-05-21 13:56:38,694 : INFO : using symmetric eta at 0.5
2019-05-21 13:56:38,696 : INFO : using serial LDA version on this node
2019-05-21 13:56:38,700 : INFO : running online (multi-pass) LDA training, 2 topics, 5 passes over the supplied corpus of 5161 documents, updating model once every 2000 documents, evaluating perplexity every 5161 documents, iterating 50x with a convergence threshold of 0.001000
2019-05-21 13:56:38,710 : INFO : PROGRESS: pass 0, at document #2000/5161
2019-05-21 13:56:39,428 : INFO : merging changes from 2000 documents into a model of 5161 documents
2019-05-21 13:56:39,435 : INFO : topic #0 (0.500): 0.001*"evenly" + 0.001*"dish" + 0.001*"condition" + 0.001*"foundationally" + 0.001*"equallyi" + 0.001*"aliveit" + 0.001*"aboriginal" + 0.001*"beboth" + 0.001*"attributerace" + 0.001*"angle"
2019-05-21 13:56:39,437 : INFO : topic #1 (0.500): 0.001*"conclude" + 0.001*"becomealso" + 0.001*"disaster" + 0.0

2019-05-21 13:56:49,363 : INFO : topic #0 (0.500): 0.000*"upwards" + 0.000*"workediting" + 0.000*"uneducated" + 0.000*"yeh" + 0.000*"standard" + 0.000*"utreasurecethis" + 0.000*"suspect" + 0.000*"stamp" + 0.000*"std" + 0.000*"steady"
2019-05-21 13:56:49,365 : INFO : topic #1 (0.500): 0.000*"liberty" + 0.000*"intelligent" + 0.000*"humansfemale" + 0.000*"older" + 0.000*"pregnancythe" + 0.000*"highlight" + 0.000*"prolifer" + 0.000*"offspring" + 0.000*"jews" + 0.000*"hominemi"
2019-05-21 13:56:49,366 : INFO : topic diff=0.458408, rho=0.389826
2019-05-21 13:56:49,701 : INFO : -9.681 per-word bound, 821.1 perplexity estimate based on a held-out corpus of 1161 documents with 1161 words
2019-05-21 13:56:49,701 : INFO : PROGRESS: pass 3, at document #5161/5161
2019-05-21 13:56:49,887 : INFO : merging changes from 1161 documents into a model of 5161 documents
2019-05-21 13:56:49,889 : INFO : topic #0 (0.500): 0.000*"upwards" + 0.000*"workediting" + 0.000*"uneducated" + 0.000*"yeh" + 0.000*"stand

In [109]:
# lets take a look at what happened
# here are the 5 most important words for each of the 2 topics we found

lda.print_topics()

2019-05-21 13:56:54,133 : INFO : topic #0 (0.500): 0.000*"upwards" + 0.000*"workediting" + 0.000*"yeh" + 0.000*"uneducated" + 0.000*"standard" + 0.000*"utreasurecethis" + 0.000*"suspect" + 0.000*"stamp" + 0.000*"std" + 0.000*"steady"
2019-05-21 13:56:54,136 : INFO : topic #1 (0.500): 0.000*"liberty" + 0.000*"humansfemale" + 0.000*"intelligent" + 0.000*"prolifer" + 0.000*"older" + 0.000*"offspring" + 0.000*"hominemi" + 0.000*"pregnancythe" + 0.000*"highlight" + 0.000*"medicaid"


[(0,
  '0.000*"upwards" + 0.000*"workediting" + 0.000*"yeh" + 0.000*"uneducated" + 0.000*"standard" + 0.000*"utreasurecethis" + 0.000*"suspect" + 0.000*"stamp" + 0.000*"std" + 0.000*"steady"'),
 (1,
  '0.000*"liberty" + 0.000*"humansfemale" + 0.000*"intelligent" + 0.000*"prolifer" + 0.000*"older" + 0.000*"offspring" + 0.000*"hominemi" + 0.000*"pregnancythe" + 0.000*"highlight" + 0.000*"medicaid"')]

In [110]:
# to map our documents to the topic space we need to actually use the LdaModel transformer that we created above

# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
lda_corpus

In [111]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

In [112]:
# Now we can take a look at the document vectors in the topic space, 
# which are measures of the component of each document along each topic.  
# Thus, at most a document vector can have num_topics=3 nonzero components in the topic space, and most have far fewer.


# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:5]

[[(0, 0.7071552), (1, 0.29284474)],
 [(0, 0.27108285), (1, 0.7289172)],
 [(0, 0.26917854), (1, 0.7308215)],
 [(0, 0.7104684), (1, 0.2895316)],
 [(0, 0.26934743), (1, 0.7306526)]]

In [ ]:
# ng_train.data[0]

# Clustering

## K-Means Clustering Algorithm

**K-means** is one of the most basic clustering algorithms.  It relies on finding cluster centers to group data points based on minimizing the sum of squared errors between each datapoint and its cluster center. 

K-means clustering is one of the most simple clustering algorithms.  One of the limitations is that it depends on the starting point of the clusters, and the number of clusters need to be defined beforehand.

In [ ]:
# helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_cluster(X,km=[],num_clusters=0):
    color = 'brgcmyk'
    alpha = 0.5
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            plt.scatter(X[km.labels_==i,0],X[km.labels_==i,1],c = color[i],alpha = alpha,s=s)
            plt.scatter(km.cluster_centers_[i][0],km.cluster_centers_[i][1],c = color[i], marker = 'x', s = 100)

In [ ]:
angle = np.linspace(0,2*np.pi,20, endpoint = False)
X = np.append([np.cos(angle)],[np.sin(angle)],0).transpose()
display_cluster(X)

#### t-SNE to visualize clusters

## DBSCAN

#### t-SNE to visualize clusters

## Heirarchical

#### t-SNE to visualize clusters

# Word 2 Vec